In [1]:
import pandas as pd
from datetime import date, datetime

## Newest Yahoo Finance Wrapper

In [2]:
# pip install yahoo_historical first 

from yahoo_historical import Fetcher

data_388 = Fetcher('0388.hk', [2007, 1, 1], [2019, 4, 18])
df_388 = data_388.getHistorical()
df_388.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-01-02,85.800003,88.300003,85.750000,88.099998,61.431114,5678100.0
1,2007-01-03,88.449997,88.449997,84.750000,86.599998,60.385193,7207900.0
2,2007-01-04,86.000000,86.000000,81.750000,83.050003,57.909805,8747300.0
3,2007-01-05,80.400002,84.000000,79.900002,82.699997,57.665775,11603400.0
4,2007-01-08,82.000000,85.500000,82.000000,84.250000,58.746578,4296000.0
5,2007-01-09,87.599998,87.599998,85.000000,85.849998,59.862225,5227500.0
6,2007-01-10,84.000000,86.699997,82.349998,82.900002,57.805244,7451900.0
7,2007-01-11,84.349998,85.000000,81.000000,81.150002,56.584969,8204100.0
8,2007-01-12,82.000000,83.400002,81.150002,82.949997,57.840092,12061400.0
9,2007-01-15,84.000000,86.349998,84.000000,86.300003,60.176010,10488100.0


In [3]:
# Zero volume days are present, have to get rid of it  

df_388[df_388['Volume'] == 0].head()

,Date,Open,High,Low,Close,Adj Close,Volume
321,2008-04-23,149.669998,149.669998,149.669998,149.669998,109.826088,0.0
326,2008-04-30,159.779999,159.779999,159.779999,159.779999,117.244705,0.0
327,2008-05-02,164.710007,164.710007,164.710007,164.710007,120.862297,0.0
329,2008-05-06,164.110001,164.110001,164.110001,164.110001,120.422012,0.0
335,2008-05-15,149.649994,149.649994,149.649994,149.649994,109.811424,0.0


In [4]:
def get_yahoo_data(symbol, market, startdate, enddate=date.today().strftime('%Y-%m-%d')):
    """
    Get  stock data from Yahoo Finance for US, HK, Singapore markets. 
    :param symbol:  HK stock symbol (4-digit number)
    :param startdate: starting yyyy-mm-dd with year, month, day in list format
    :param enddate: endng yyyy-mm-dd with year, month, day in list format
    :return:  dataframe without zero volume days.
    """
    dict_suffix = {'us': '', 'hk': '.hk', 'sg': '.SI'}
    dt_startdate = datetime.strptime(startdate, '%Y-%m-%d')
    dt_enddate = datetime.strptime(enddate, '%Y-%m-%d')
    start_date = [dt_startdate.year, dt_startdate.month, dt_startdate.day]
    end_date = [dt_enddate.year, dt_enddate.month, dt_enddate.day]
    data = Fetcher(symbol + dict_suffix[market], start_date, end_date)
    df = data.getHistorical()
    df = df[df['Volume'] > 0]
    df.set_index('Date', inplace=True)
    return df
    
df_0005 = get_yahoo_data('0005', 'hk', '2000-01-01', date.today().strftime('%Y-%m-%d'))
df_0005.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-04-12,66.599998,66.900002,66.500000,66.900002,66.900002,11869128.0
2019-04-15,67.500000,67.800003,67.099998,67.099998,67.099998,20520188.0
2019-04-16,66.800003,67.699997,66.800003,67.599998,67.599998,14092313.0
2019-04-17,67.699997,67.949997,67.449997,67.849998,67.849998,17999581.0
2019-04-18,67.949997,68.000000,67.550003,67.599998,67.599998,14176761.0


## From IEX Data

In [5]:
from iexfinance.stocks import get_historical_data

startdate = datetime(2015, 1, 1)
enddate = date.today()
iex_aapl = get_historical_data('AAPL', startdate, enddate, output_format='pandas')

iex_aapl.head()

,open,high,low,close,volume
date,,,,,
2015-01-02,103.4410,103.4874,99.6893,101.5280,53204626
2015-01-05,100.5622,100.8965,97.8877,98.6678,64285491
2015-01-06,98.9371,99.7636,97.1634,98.6771,65797116
2015-01-07,99.5500,100.4786,99.0810,100.0607,40105934
2015-01-08,101.4351,104.1468,100.9429,103.9053,59364547


In [6]:
def get_iex_dict(symbol_list, startdate):
    """Get dictionary of dataframes from IEX"""
    dict_list = dict()
    startdate = datetime.strptime(startdate, '%Y-%m-%d')
    for symbol in symbol_list:
        df = get_historical_data(symbol, startdate, date.today(), output_format='pandas')
        dict_list[symbol] = df
    return dict_list

symbol_list = ['AAPL', 'AMZN', 'NFLX', 'INTC', 'SPY', 'QQQ']
iex_dict = get_iex_dict(symbol_list, '2015-01-01')
iex_dict['AMZN'].tail()

,open,high,low,close,volume
date,,,,,
2019-04-12,1848.40,1851.50,1841.30,1843.06,3114413
2019-04-15,1842.00,1846.85,1818.90,1844.87,3724423
2019-04-16,1851.35,1869.77,1848.00,1863.04,3044618
2019-04-17,1872.99,1876.47,1860.44,1864.82,2893517
2019-04-18,1868.79,1870.82,1859.48,1861.69,2749882


In [9]:
from iexfinance.stocks import get_historical_intraday

iex_aapl_itd = get_historical_intraday("AAPL", startdate='2019-04-01', output_format='pandas')
print(iex_aapl_itd.head(15))

                     average  changeOverTime    close      date     high  \
2019-04-18 09:30:00  203.207        0.000000  203.190  20190418  203.360   
2019-04-18 09:31:00  203.317        0.000541  203.330  20190418  203.480   
2019-04-18 09:32:00  203.124       -0.000408  203.220  20190418  203.340   
2019-04-18 09:33:00  203.248        0.000202  203.390  20190418  203.460   
2019-04-18 09:34:00  203.556        0.001717  203.440  20190418  203.650   
2019-04-18 09:35:00  203.669        0.002274  203.825  20190418  203.825   
2019-04-18 09:36:00  203.618        0.002023  203.525  20190418  203.830   
2019-04-18 09:37:00  203.456        0.001225  203.390  20190418  203.570   
2019-04-18 09:38:00  203.222        0.000074  203.020  20190418  203.450   
2019-04-18 09:39:00  202.952       -0.001255  202.970  20190418  203.140   
2019-04-18 09:40:00  202.994       -0.001048  203.130  20190418  203.130   
2019-04-18 09:41:00  203.280        0.000359  203.340  20190418  203.360   
2019-04-18 0

In [10]:
iex_aapl_itd.shape

(390, 20)

## AlphaVantage (HTTP Request)

In [11]:
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=2318.HK&apikey=[yourAPIKey]'
response = urlopen(url)
json_2318 = response.read()

data_2318 = json.loads(json_2318)
df_2318 = pd.DataFrame.from_dict(data_2318['Time Series (Daily)'])
df_2318 = df_2318.T
df_2318.index = pd.to_datetime(df_2318.index)
print(df_2318.head())

            1. open  2. high   3. low 4. close 5. volume
2019-04-18  95.6500  95.6500  94.1000  94.5000  24229379
2019-04-17  94.7500  95.6500  93.7000  94.4500  30245080
2019-04-16  92.5000  95.1000  92.1500  94.7500  35753933
2019-04-15  93.9500  95.2500  92.5500  92.6000  40867262
2019-04-12  91.7000  92.6500  91.3000  92.6500  23123193


In [22]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=QQQ&interval=5min&apikey=[yourAPIKey]'
response = urlopen(url)
json_qqq = response.read()

data_qqq = json.loads(json_qqq)
df_qqq = pd.DataFrame.from_dict(data_qqq['Time Series (5min)'])
df_qqq = df_qqq.T
df_qqq.index = pd.to_datetime(df_qqq.index)
print(df_qqq.tail())



                      1. open   2. high    3. low  4. close 5. volume
2019-04-17 14:35:00  186.6700  186.8400  186.6700  186.8100    342736
2019-04-17 14:30:00  186.8300  186.8425  186.6000  186.6600    368291
2019-04-17 14:25:00  187.0700  187.0800  186.8050  186.8400    215501
2019-04-17 14:20:00  186.8400  187.0900  186.8400  187.0600    414414
2019-04-17 14:15:00  186.9600  186.9800  186.7600  186.8400    313061


## Tushare (For Chinese A & B Shares)

### Individual stock EOD data

In [14]:
import tushare as ts

pro = ts.pro_api('api_token')
df_0036 = pro.daily(ts_code='600036.SH', start_date='20140101', end_date='20190418')
print(df_0036.tail(10))

        ts_code trade_date   open   high    low  close  pre_close  change  \
1276  600036.SH   20140115  10.72  10.75  10.53  10.59      10.75   -0.16   
1277  600036.SH   20140114  10.85  10.92  10.65  10.75      10.88   -0.13   
1278  600036.SH   20140113  10.82  10.98  10.73  10.88      10.80    0.08   
1279  600036.SH   20140110  10.69  10.82  10.61  10.80      10.70    0.10   
1280  600036.SH   20140109  10.61  10.76  10.61  10.70      10.63    0.07   
1281  600036.SH   20140108  10.52  10.69  10.46  10.63      10.52    0.11   
1282  600036.SH   20140107  10.33  10.54  10.28  10.52      10.46    0.06   
1283  600036.SH   20140106  10.50  10.52  10.32  10.46      10.51   -0.05   
1284  600036.SH   20140103  10.65  10.68  10.40  10.51      10.73   -0.22   
1285  600036.SH   20140102  10.84  10.86  10.67  10.73      10.89   -0.16   

      pct_chg        vol      amount  
1276    -1.49  355733.95  377633.071  
1277    -1.19  448678.87  482287.733  
1278     0.74  779716.00  848646.19

### Print list of all Shanghai-Connect stocks

In [15]:
pro = ts.pro_api('api-token')
df_components = pro.hs_const(hs_type='SH')
print(df_components.head(10), '\n')
print(df_components.tail(10))

     ts_code hs_type   in_date out_date is_new
0  603818.SH      SH  20160613     None      1
1  603108.SH      SH  20161212     None      1
2  600507.SH      SH  20141117     None      1
3  601377.SH      SH  20141117     None      1
4  600309.SH      SH  20141117     None      1
5  600298.SH      SH  20141117     None      1
6  600018.SH      SH  20141117     None      1
7  600483.SH      SH  20151214     None      1
8  600068.SH      SH  20141117     None      1
9  600594.SH      SH  20141117     None      1 

       ts_code hs_type   in_date out_date is_new
567  601015.SH      SH  20150615     None      1
568  600557.SH      SH  20141117     None      1
569  603118.SH      SH  20151214     None      1
570  600988.SH      SH  20150615     None      1
571  603306.SH      SH  20150521     None      1
572  601000.SH      SH  20141117     None      1
573  600171.SH      SH  20141117     None      1
574  600422.SH      SH  20141117     None      1
575  601991.SH      SH  20141117     Non

### From STOOQ (for Futures & Forex)

In [19]:
import os
import time
from selenium import webdriver
browser = webdriver.Chrome(os.path.join(os.path.join('D:\\', 'Will', 'chromedriver.exe')))

import shutil
download_path = os.path.join(os.path.expanduser('~'), 'Downloads')

def get_stooq_move(symbol_list, download_path, csv_path, type):
    dict_link = {'indices': '', 'futures': '.F', 'forex': ''}
    dict_csv = {'indices': '_d', 'futures': '_f_d', 'forex': '_d'}
    for ticker in symbol_list:
        # Use Selenium Chrome to click to download CSV (urlretrieve NOT working well)
        url = 'https://stooq.com/q/d/?s=' + ticker + dict_link[type]
        browser.get(url)
        link_csv = browser.find_element_by_link_text('Download data in csv file...')
        link_csv.click()
        time.sleep(1)
        # Move the file from download folder to own target folder
        target_file = os.path.join(csv_path, ticker + '_' + date.today().strftime('%Y-%m-%d') + '.csv')
        shutil.move(os.path.join(download_path, ticker.lower() + dict_csv[type] + '.csv'), target_file)
    browser.close()

csv_path = os.path.join(os.path.abspath('csv_STOOQ'))
if not os.path.exists(csv_path):
    os.makedirs(csv_path)

indices_list = ['^HSI', '^SPX', '^DJI', '^DAX', '^NKX']
futures_list = ['ES', 'NQ', 'YM', 'RJ', 'GC', 'CL']
forex_list = ['EURUSD', 'GBPUSD', 'USDJPY', 'USDCAD', 'EURJPY', 'GBPJPY']

get_stooq_move(indices_list, download_path, csv_path, 'indices')
get_stooq_move(futures_list, download_path, csv_path, 'futures')
get_stooq_move(forex_list, download_path, csv_path, 'forex')


In [21]:
df_es = pd.read_csv(os.path.join(csv_path, '^spx_' + date.today().strftime('%Y-%m-%d') + '.csv'), header=0, index_col='Date', parse_dates=True)
print(df_es.loc['1987-10-19':].head())

              Open    High     Low   Close       Volume
Date                                                   
1987-10-19  282.70  282.70  224.83  224.84  671444433.0
1987-10-20  225.06  245.62  216.46  236.83  675666679.0
1987-10-21  236.83  259.27  236.83  258.38  499555556.0
1987-10-22  258.24  258.38  242.99  248.25  435777801.0
1987-10-23  248.29  250.70  242.76  248.22  272888889.0
